In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 16
tree_depth = 12
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [3]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [4]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [5]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [6]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.197713851928711 | KNN Loss: 6.226964950561523 | BCE Loss: 1.9707493782043457
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.225666999816895 | KNN Loss: 6.226774215698242 | BCE Loss: 1.9988924264907837
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.174530029296875 | KNN Loss: 6.2269487380981445 | BCE Loss: 1.94758141040802
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.190489768981934 | KNN Loss: 6.226901054382324 | BCE Loss: 1.9635889530181885
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.177996635437012 | KNN Loss: 6.226274013519287 | BCE Loss: 1.9517223834991455
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.151252746582031 | KNN Loss: 6.225853443145752 | BCE Loss: 1.9253997802734375
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.15381145477295 | KNN Loss: 6.225696086883545 | BCE Loss: 1.9281151294708252
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.157215118408203 | KNN Loss: 6.225313186645508 | BCE Loss: 1.9319019

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 6.8714399337768555 | KNN Loss: 5.739673137664795 | BCE Loss: 1.13176691532135
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 6.798647880554199 | KNN Loss: 5.666825771331787 | BCE Loss: 1.131821870803833
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 6.681827545166016 | KNN Loss: 5.56065034866333 | BCE Loss: 1.1211771965026855
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 6.610039710998535 | KNN Loss: 5.499035835266113 | BCE Loss: 1.1110038757324219
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 6.4639201164245605 | KNN Loss: 5.373536109924316 | BCE Loss: 1.0903841257095337
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 6.341156959533691 | KNN Loss: 5.257620811462402 | BCE Loss: 1.0835363864898682
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 6.213466644287109 | KNN Loss: 5.108219623565674 | BCE Loss: 1.1052470207214355
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 6.144377708435059 | KNN Loss: 5.05210018157959 | BCE Loss: 1

Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 4.320324897766113 | KNN Loss: 3.2698252201080322 | BCE Loss: 1.050499439239502
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 4.358704090118408 | KNN Loss: 3.313469409942627 | BCE Loss: 1.0452346801757812
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 4.304399013519287 | KNN Loss: 3.252042055130005 | BCE Loss: 1.0523569583892822
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 4.2353925704956055 | KNN Loss: 3.204836130142212 | BCE Loss: 1.030556559562683
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 4.277061462402344 | KNN Loss: 3.227719306945801 | BCE Loss: 1.0493419170379639
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 4.300971031188965 | KNN Loss: 3.247703790664673 | BCE Loss: 1.053267240524292
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 4.378009796142578 | KNN Loss: 3.311530113220215 | BCE Loss: 1.0664796829223633
Epoch 22 / 500 | iteration 25 / 30 | Total Loss: 4.334157943725586 | KNN Loss: 3.281982421875 | BCE Loss: 1

Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 4.237036228179932 | KNN Loss: 3.198315143585205 | BCE Loss: 1.038720965385437
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 4.232545852661133 | KNN Loss: 3.216519832611084 | BCE Loss: 1.016026258468628
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 4.190826892852783 | KNN Loss: 3.1581554412841797 | BCE Loss: 1.0326714515686035
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 4.197216987609863 | KNN Loss: 3.1687138080596924 | BCE Loss: 1.02850341796875
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 4.229626655578613 | KNN Loss: 3.184157609939575 | BCE Loss: 1.045469045639038
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 4.245745658874512 | KNN Loss: 3.210498571395874 | BCE Loss: 1.0352468490600586
Epoch 33 / 500 | iteration 15 / 30 | Total Loss: 4.27773380279541 | KNN Loss: 3.2555296421051025 | BCE Loss: 1.0222043991088867
Epoch 33 / 500 | iteration 20 / 30 | Total Loss: 4.198144435882568 | KNN Loss: 3.184840440750122 | BCE Loss: 

Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 4.156114101409912 | KNN Loss: 3.1511781215667725 | BCE Loss: 1.0049360990524292
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 4.241332530975342 | KNN Loss: 3.19590163230896 | BCE Loss: 1.0454308986663818
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 4.158176422119141 | KNN Loss: 3.1401865482330322 | BCE Loss: 1.0179896354675293
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 4.214005470275879 | KNN Loss: 3.1279385089874268 | BCE Loss: 1.0860671997070312
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 4.188873291015625 | KNN Loss: 3.1579582691192627 | BCE Loss: 1.0309152603149414
Epoch 44 / 500 | iteration 5 / 30 | Total Loss: 4.160538196563721 | KNN Loss: 3.1324663162231445 | BCE Loss: 1.0280717611312866
Epoch 44 / 500 | iteration 10 / 30 | Total Loss: 4.171976089477539 | KNN Loss: 3.175236225128174 | BCE Loss: 0.9967400431632996
Epoch 44 / 500 | iteration 15 / 30 | Total Loss: 4.1338043212890625 | KNN Loss: 3.134368419647217 | BC

...

Epoch 449 / 500 | iteration 0 / 30 | Total Loss: 4.074820518493652 | KNN Loss: 3.059860944747925 | BCE Loss: 1.0149593353271484
Epoch 449 / 500 | iteration 5 / 30 | Total Loss: 4.077322959899902 | KNN Loss: 3.0536935329437256 | BCE Loss: 1.0236293077468872
Epoch 449 / 500 | iteration 10 / 30 | Total Loss: 4.085323333740234 | KNN Loss: 3.068756341934204 | BCE Loss: 1.0165669918060303
Epoch 449 / 500 | iteration 15 / 30 | Total Loss: 4.125814437866211 | KNN Loss: 3.0888099670410156 | BCE Loss: 1.0370047092437744
Epoch 449 / 500 | iteration 20 / 30 | Total Loss: 4.084348678588867 | KNN Loss: 3.0829086303710938 | BCE Loss: 1.0014402866363525
Epoch 449 / 500 | iteration 25 / 30 | Total Loss: 4.132314682006836 | KNN Loss: 3.0944225788116455 | BCE Loss: 1.0378918647766113
Epoch 450 / 500 | iteration 0 / 30 | Total Loss: 4.035597324371338 | KNN Loss: 3.0204126834869385 | BCE Loss: 1.0151846408843994
Epoch 450 / 500 | iteration 5 / 30 | Total Loss: 4.066457748413086 | KNN Loss: 3.07345724105834

Epoch 459 / 500 | iteration 20 / 30 | Total Loss: 4.079638957977295 | KNN Loss: 3.042768955230713 | BCE Loss: 1.0368698835372925
Epoch 459 / 500 | iteration 25 / 30 | Total Loss: 4.109965801239014 | KNN Loss: 3.082084894180298 | BCE Loss: 1.0278807878494263
Epoch 460 / 500 | iteration 0 / 30 | Total Loss: 4.082767486572266 | KNN Loss: 3.072995662689209 | BCE Loss: 1.0097715854644775
Epoch 460 / 500 | iteration 5 / 30 | Total Loss: 4.054298400878906 | KNN Loss: 3.0378551483154297 | BCE Loss: 1.0164430141448975
Epoch 460 / 500 | iteration 10 / 30 | Total Loss: 4.093414783477783 | KNN Loss: 3.0742766857147217 | BCE Loss: 1.019137978553772
Epoch 460 / 500 | iteration 15 / 30 | Total Loss: 4.090819835662842 | KNN Loss: 3.07895827293396 | BCE Loss: 1.0118614435195923
Epoch 460 / 500 | iteration 20 / 30 | Total Loss: 4.145552158355713 | KNN Loss: 3.1148338317871094 | BCE Loss: 1.030718207359314
Epoch 460 / 500 | iteration 25 / 30 | Total Loss: 4.0599870681762695 | KNN Loss: 3.039482355117798 

Epoch 470 / 500 | iteration 10 / 30 | Total Loss: 4.104984283447266 | KNN Loss: 3.056821823120117 | BCE Loss: 1.0481626987457275
Epoch 470 / 500 | iteration 15 / 30 | Total Loss: 4.079991340637207 | KNN Loss: 3.051877498626709 | BCE Loss: 1.0281140804290771
Epoch 470 / 500 | iteration 20 / 30 | Total Loss: 4.057466983795166 | KNN Loss: 3.0654470920562744 | BCE Loss: 0.9920198917388916
Epoch 470 / 500 | iteration 25 / 30 | Total Loss: 4.107001781463623 | KNN Loss: 3.0615482330322266 | BCE Loss: 1.045453429222107
Epoch 471 / 500 | iteration 0 / 30 | Total Loss: 4.0921549797058105 | KNN Loss: 3.0709269046783447 | BCE Loss: 1.0212280750274658
Epoch 471 / 500 | iteration 5 / 30 | Total Loss: 4.114859580993652 | KNN Loss: 3.079730749130249 | BCE Loss: 1.0351288318634033
Epoch 471 / 500 | iteration 10 / 30 | Total Loss: 4.035788059234619 | KNN Loss: 3.0500328540802 | BCE Loss: 0.985755205154419
Epoch 471 / 500 | iteration 15 / 30 | Total Loss: 4.117266654968262 | KNN Loss: 3.072608470916748 |

Epoch 481 / 500 | iteration 0 / 30 | Total Loss: 4.073573589324951 | KNN Loss: 3.070242404937744 | BCE Loss: 1.003331184387207
Epoch 481 / 500 | iteration 5 / 30 | Total Loss: 4.109986782073975 | KNN Loss: 3.0789897441864014 | BCE Loss: 1.0309971570968628
Epoch 481 / 500 | iteration 10 / 30 | Total Loss: 4.059248924255371 | KNN Loss: 3.0500409603118896 | BCE Loss: 1.0092079639434814
Epoch 481 / 500 | iteration 15 / 30 | Total Loss: 4.138001918792725 | KNN Loss: 3.11407732963562 | BCE Loss: 1.023924708366394
Epoch 481 / 500 | iteration 20 / 30 | Total Loss: 4.1047563552856445 | KNN Loss: 3.072605609893799 | BCE Loss: 1.0321505069732666
Epoch 481 / 500 | iteration 25 / 30 | Total Loss: 4.089522838592529 | KNN Loss: 3.062883138656616 | BCE Loss: 1.026639699935913
Epoch 482 / 500 | iteration 0 / 30 | Total Loss: 4.08200740814209 | KNN Loss: 3.0660314559936523 | BCE Loss: 1.0159759521484375
Epoch 482 / 500 | iteration 5 / 30 | Total Loss: 4.118971824645996 | KNN Loss: 3.0805106163024902 | B

Epoch 491 / 500 | iteration 20 / 30 | Total Loss: 4.110325813293457 | KNN Loss: 3.0628061294555664 | BCE Loss: 1.0475199222564697
Epoch 491 / 500 | iteration 25 / 30 | Total Loss: 4.0865702629089355 | KNN Loss: 3.091723918914795 | BCE Loss: 0.9948463439941406
Epoch 492 / 500 | iteration 0 / 30 | Total Loss: 4.10693359375 | KNN Loss: 3.0725910663604736 | BCE Loss: 1.0343425273895264
Epoch 492 / 500 | iteration 5 / 30 | Total Loss: 4.1229658126831055 | KNN Loss: 3.0546011924743652 | BCE Loss: 1.0683645009994507
Epoch 492 / 500 | iteration 10 / 30 | Total Loss: 4.117815971374512 | KNN Loss: 3.0638058185577393 | BCE Loss: 1.0540101528167725
Epoch 492 / 500 | iteration 15 / 30 | Total Loss: 4.043185710906982 | KNN Loss: 3.0612683296203613 | BCE Loss: 0.9819175004959106
Epoch 492 / 500 | iteration 20 / 30 | Total Loss: 4.104577541351318 | KNN Loss: 3.0563957691192627 | BCE Loss: 1.0481818914413452
Epoch 492 / 500 | iteration 25 / 30 | Total Loss: 4.127537727355957 | KNN Loss: 3.0996971130371

In [7]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 2.9690,  3.3708,  2.5624,  3.2352,  3.2558,  0.8220,  2.5128,  2.0602,
          2.2463,  2.0747,  2.2316,  2.1456,  0.8478,  1.8535,  1.3311,  1.4486,
          2.7928,  2.8470,  2.7278,  2.3734,  1.8174,  2.6831,  2.3625,  2.3584,
          2.3992,  1.5953,  1.8204,  1.4668,  1.5452,  0.4173, -0.2491,  1.0625,
          0.1785,  0.9827,  1.6118,  1.5006,  0.9838,  2.9433,  0.8661,  1.3627,
          1.0244, -0.8252, -0.2011,  2.2390,  2.2572,  0.7698, -0.1759,  0.1700,
          1.4033,  2.3332,  1.8797,  0.0729,  1.3771,  0.5723, -0.6208,  1.2612,
          0.9353,  1.3836,  1.3940,  1.8973,  0.7152,  0.9429,  0.3115,  1.7001,
          1.3407,  1.5566, -1.8287,  0.3413,  2.3174,  2.1991,  2.4107,  0.5554,
          1.3664,  2.2806,  2.0282,  1.3582,  0.1327,  0.7336,  0.2443,  1.6848,
          0.0856,  0.4232,  1.6518, -0.3664,  0.2874, -1.0850, -2.3750, -0.1772,
          0.6430, -1.8912,  0.5034, -0.2220, -0.5476, -0.9107,  0.6913,  1.2768,
         -0.6644, -0.7303,  

In [8]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [10]:
dataset_ = [d[0].cpu() for d in dataset]

In [11]:
model = model.eval().cpu()
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:00<00:00, 81.65it/s]


In [12]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [13]:
clusters = DBSCAN(eps=0.2, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [14]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [16]:
tensor_dataset = torch.stack(dataset_)

In [17]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [18]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [19]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [20]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [21]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [22]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [23]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [24]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [25]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [26]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [27]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [28]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [29]:
losses = []
accs = []
sparsity = []

In [30]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
layer 7: 0.0
layer 8: 0.0
layer 9: 0.0
layer 10: 0.0
Epoch: 00 | Batch: 000 / 018 | Total loss: 9.632 | Reg loss: 0.014 | Tree loss: 9.632 | Accuracy: 0.000000 | 4.111 sec/iter
Epoch: 00 | Batch: 001 / 018 | Total loss: 9.630 | Reg loss: 0.013 | Tree loss: 9.630 | Accuracy: 0.000000 | 4.112 sec/iter
Epoch: 00 | Batch: 002 / 018 | Total loss: 9.630 | Reg loss: 0.012 | Tree loss: 9.630 | Accuracy: 0.007812 | 4.084 sec/iter
Epoch: 00 | Batch: 003 / 018 | Total loss: 9.629 | Reg loss: 0.011 | Tree loss: 9.629 | Accuracy: 0.035156 | 4.089 sec/iter
Epoch: 00 | Batch: 004 / 018 | Total loss: 9.627 | Reg loss: 0.010 | Tree loss: 9.627 | Accuracy: 0.060547 | 4.094 sec/iter
Epoch: 00 | Batch: 005 / 018 | Total loss: 9.625 | Reg loss: 0.009 | Tree loss: 9.625 | Accuracy: 0.062500 | 4.102 sec/iter
Epoch: 00 | Batch: 006 / 018 | Total loss: 9.625 | Reg loss: 0.008 | Tree loss: 9.625 | 

Epoch: 03 | Batch: 003 / 018 | Total loss: 9.602 | Reg loss: 0.005 | Tree loss: 9.602 | Accuracy: 0.074219 | 4.972 sec/iter
Epoch: 03 | Batch: 004 / 018 | Total loss: 9.602 | Reg loss: 0.005 | Tree loss: 9.602 | Accuracy: 0.076172 | 4.972 sec/iter
Epoch: 03 | Batch: 005 / 018 | Total loss: 9.600 | Reg loss: 0.005 | Tree loss: 9.600 | Accuracy: 0.080078 | 4.967 sec/iter
Epoch: 03 | Batch: 006 / 018 | Total loss: 9.599 | Reg loss: 0.005 | Tree loss: 9.599 | Accuracy: 0.058594 | 4.966 sec/iter
Epoch: 03 | Batch: 007 / 018 | Total loss: 9.597 | Reg loss: 0.005 | Tree loss: 9.597 | Accuracy: 0.072266 | 4.965 sec/iter
Epoch: 03 | Batch: 008 / 018 | Total loss: 9.598 | Reg loss: 0.005 | Tree loss: 9.598 | Accuracy: 0.080078 | 4.964 sec/iter
Epoch: 03 | Batch: 009 / 018 | Total loss: 9.597 | Reg loss: 0.006 | Tree loss: 9.597 | Accuracy: 0.082031 | 4.962 sec/iter
Epoch: 03 | Batch: 010 / 018 | Total loss: 9.597 | Reg loss: 0.006 | Tree loss: 9.597 | Accuracy: 0.070312 | 4.958 sec/iter
Epoch: 0

Epoch: 06 | Batch: 007 / 018 | Total loss: 9.576 | Reg loss: 0.009 | Tree loss: 9.576 | Accuracy: 0.052734 | 5.064 sec/iter
Epoch: 06 | Batch: 008 / 018 | Total loss: 9.569 | Reg loss: 0.009 | Tree loss: 9.569 | Accuracy: 0.085938 | 5.063 sec/iter
Epoch: 06 | Batch: 009 / 018 | Total loss: 9.576 | Reg loss: 0.009 | Tree loss: 9.576 | Accuracy: 0.076172 | 5.061 sec/iter
Epoch: 06 | Batch: 010 / 018 | Total loss: 9.564 | Reg loss: 0.009 | Tree loss: 9.564 | Accuracy: 0.064453 | 5.058 sec/iter
Epoch: 06 | Batch: 011 / 018 | Total loss: 9.567 | Reg loss: 0.009 | Tree loss: 9.567 | Accuracy: 0.087891 | 5.055 sec/iter
Epoch: 06 | Batch: 012 / 018 | Total loss: 9.565 | Reg loss: 0.010 | Tree loss: 9.565 | Accuracy: 0.062500 | 5.052 sec/iter
Epoch: 06 | Batch: 013 / 018 | Total loss: 9.563 | Reg loss: 0.010 | Tree loss: 9.563 | Accuracy: 0.074219 | 5.049 sec/iter
Epoch: 06 | Batch: 014 / 018 | Total loss: 9.557 | Reg loss: 0.010 | Tree loss: 9.557 | Accuracy: 0.064453 | 5.046 sec/iter
Epoch: 0

Epoch: 09 | Batch: 011 / 018 | Total loss: 9.366 | Reg loss: 0.013 | Tree loss: 9.366 | Accuracy: 0.074219 | 5.085 sec/iter
Epoch: 09 | Batch: 012 / 018 | Total loss: 9.355 | Reg loss: 0.014 | Tree loss: 9.355 | Accuracy: 0.076172 | 5.082 sec/iter
Epoch: 09 | Batch: 013 / 018 | Total loss: 9.355 | Reg loss: 0.014 | Tree loss: 9.355 | Accuracy: 0.054688 | 5.08 sec/iter
Epoch: 09 | Batch: 014 / 018 | Total loss: 9.343 | Reg loss: 0.014 | Tree loss: 9.343 | Accuracy: 0.076172 | 5.078 sec/iter
Epoch: 09 | Batch: 015 / 018 | Total loss: 9.317 | Reg loss: 0.015 | Tree loss: 9.317 | Accuracy: 0.082031 | 5.076 sec/iter
Epoch: 09 | Batch: 016 / 018 | Total loss: 9.317 | Reg loss: 0.015 | Tree loss: 9.317 | Accuracy: 0.074219 | 5.075 sec/iter
Epoch: 09 | Batch: 017 / 018 | Total loss: 9.294 | Reg loss: 0.015 | Tree loss: 9.294 | Accuracy: 0.073171 | 5.066 sec/iter
Average sparseness: 0.9821428571428572
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0

Epoch: 12 | Batch: 015 / 018 | Total loss: 8.846 | Reg loss: 0.017 | Tree loss: 8.846 | Accuracy: 0.064453 | 5.059 sec/iter
Epoch: 12 | Batch: 016 / 018 | Total loss: 8.819 | Reg loss: 0.017 | Tree loss: 8.819 | Accuracy: 0.099609 | 5.058 sec/iter
Epoch: 12 | Batch: 017 / 018 | Total loss: 8.788 | Reg loss: 0.017 | Tree loss: 8.788 | Accuracy: 0.121951 | 5.052 sec/iter
Average sparseness: 0.9821428571428572
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
layer 9: 0.9821428571428573
layer 10: 0.9821428571428573
Epoch: 13 | Batch: 000 / 018 | Total loss: 8.983 | Reg loss: 0.015 | Tree loss: 8.983 | Accuracy: 0.076172 | 5.068 sec/iter
Epoch: 13 | Batch: 001 / 018 | Total loss: 8.941 | Reg loss: 0.015 | Tree loss: 8.941 | Accuracy: 0.078125 | 5.07 sec/iter
Epoch: 13 | Batch: 002 / 018 | Tot

...

Epoch: 86 | Batch: 008 / 018 | Total loss: 3.694 | Reg loss: 0.023 | Tree loss: 3.694 | Accuracy: 0.070312 | 5.131 sec/iter
Epoch: 86 | Batch: 009 / 018 | Total loss: 3.767 | Reg loss: 0.023 | Tree loss: 3.767 | Accuracy: 0.085938 | 5.132 sec/iter
Epoch: 86 | Batch: 010 / 018 | Total loss: 3.633 | Reg loss: 0.023 | Tree loss: 3.633 | Accuracy: 0.103516 | 5.132 sec/iter
Epoch: 86 | Batch: 011 / 018 | Total loss: 3.693 | Reg loss: 0.023 | Tree loss: 3.693 | Accuracy: 0.068359 | 5.132 sec/iter
Epoch: 86 | Batch: 012 / 018 | Total loss: 3.641 | Reg loss: 0.023 | Tree loss: 3.641 | Accuracy: 0.082031 | 5.132 sec/iter
Epoch: 86 | Batch: 013 / 018 | Total loss: 3.758 | Reg loss: 0.023 | Tree loss: 3.758 | Accuracy: 0.076172 | 5.132 sec/iter
Epoch: 86 | Batch: 014 / 018 | Total loss: 3.727 | Reg loss: 0.023 | Tree loss: 3.727 | Accuracy: 0.076172 | 5.132 sec/iter
Epoch: 86 | Batch: 015 / 018 | Total loss: 3.695 | Reg loss: 0.023 | Tree loss: 3.695 | Accuracy: 0.091797 | 5.132 sec/iter
Epoch: 8

Epoch: 89 | Batch: 012 / 018 | Total loss: 3.689 | Reg loss: 0.024 | Tree loss: 3.689 | Accuracy: 0.085938 | 5.13 sec/iter
Epoch: 89 | Batch: 013 / 018 | Total loss: 3.696 | Reg loss: 0.024 | Tree loss: 3.696 | Accuracy: 0.085938 | 5.13 sec/iter
Epoch: 89 | Batch: 014 / 018 | Total loss: 3.656 | Reg loss: 0.024 | Tree loss: 3.656 | Accuracy: 0.083984 | 5.129 sec/iter
Epoch: 89 | Batch: 015 / 018 | Total loss: 3.639 | Reg loss: 0.024 | Tree loss: 3.639 | Accuracy: 0.082031 | 5.129 sec/iter
Epoch: 89 | Batch: 016 / 018 | Total loss: 3.719 | Reg loss: 0.024 | Tree loss: 3.719 | Accuracy: 0.082031 | 5.129 sec/iter
Epoch: 89 | Batch: 017 / 018 | Total loss: 3.680 | Reg loss: 0.024 | Tree loss: 3.680 | Accuracy: 0.195122 | 5.128 sec/iter
Average sparseness: 0.9821428571428572
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428

Epoch: 92 | Batch: 016 / 018 | Total loss: 3.661 | Reg loss: 0.024 | Tree loss: 3.661 | Accuracy: 0.078125 | 5.136 sec/iter
Epoch: 92 | Batch: 017 / 018 | Total loss: 3.719 | Reg loss: 0.024 | Tree loss: 3.719 | Accuracy: 0.024390 | 5.135 sec/iter
Average sparseness: 0.9821428571428572
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
layer 9: 0.9821428571428573
layer 10: 0.9821428571428573
Epoch: 93 | Batch: 000 / 018 | Total loss: 3.696 | Reg loss: 0.024 | Tree loss: 3.696 | Accuracy: 0.076172 | 5.136 sec/iter
Epoch: 93 | Batch: 001 / 018 | Total loss: 3.717 | Reg loss: 0.024 | Tree loss: 3.717 | Accuracy: 0.093750 | 5.136 sec/iter
Epoch: 93 | Batch: 002 / 018 | Total loss: 3.706 | Reg loss: 0.024 | Tree loss: 3.706 | Accuracy: 0.085938 | 5.136 sec/iter
Epoch: 93 | Batch: 003 / 018 | To

Epoch: 96 | Batch: 000 / 018 | Total loss: 3.744 | Reg loss: 0.024 | Tree loss: 3.744 | Accuracy: 0.080078 | 5.142 sec/iter
Epoch: 96 | Batch: 001 / 018 | Total loss: 3.639 | Reg loss: 0.024 | Tree loss: 3.639 | Accuracy: 0.105469 | 5.142 sec/iter
Epoch: 96 | Batch: 002 / 018 | Total loss: 3.680 | Reg loss: 0.024 | Tree loss: 3.680 | Accuracy: 0.083984 | 5.142 sec/iter
Epoch: 96 | Batch: 003 / 018 | Total loss: 3.692 | Reg loss: 0.024 | Tree loss: 3.692 | Accuracy: 0.074219 | 5.143 sec/iter
Epoch: 96 | Batch: 004 / 018 | Total loss: 3.705 | Reg loss: 0.024 | Tree loss: 3.705 | Accuracy: 0.072266 | 5.143 sec/iter
Epoch: 96 | Batch: 005 / 018 | Total loss: 3.701 | Reg loss: 0.024 | Tree loss: 3.701 | Accuracy: 0.072266 | 5.143 sec/iter
Epoch: 96 | Batch: 006 / 018 | Total loss: 3.624 | Reg loss: 0.024 | Tree loss: 3.624 | Accuracy: 0.093750 | 5.143 sec/iter
Epoch: 96 | Batch: 007 / 018 | Total loss: 3.628 | Reg loss: 0.024 | Tree loss: 3.628 | Accuracy: 0.078125 | 5.144 sec/iter
Epoch: 9

Epoch: 99 | Batch: 004 / 018 | Total loss: 3.672 | Reg loss: 0.024 | Tree loss: 3.672 | Accuracy: 0.080078 | 5.145 sec/iter
Epoch: 99 | Batch: 005 / 018 | Total loss: 3.619 | Reg loss: 0.024 | Tree loss: 3.619 | Accuracy: 0.072266 | 5.145 sec/iter
Epoch: 99 | Batch: 006 / 018 | Total loss: 3.718 | Reg loss: 0.024 | Tree loss: 3.718 | Accuracy: 0.089844 | 5.145 sec/iter
Epoch: 99 | Batch: 007 / 018 | Total loss: 3.681 | Reg loss: 0.024 | Tree loss: 3.681 | Accuracy: 0.082031 | 5.145 sec/iter
Epoch: 99 | Batch: 008 / 018 | Total loss: 3.689 | Reg loss: 0.024 | Tree loss: 3.689 | Accuracy: 0.085938 | 5.146 sec/iter
Epoch: 99 | Batch: 009 / 018 | Total loss: 3.727 | Reg loss: 0.024 | Tree loss: 3.727 | Accuracy: 0.085938 | 5.146 sec/iter
Epoch: 99 | Batch: 010 / 018 | Total loss: 3.691 | Reg loss: 0.024 | Tree loss: 3.691 | Accuracy: 0.064453 | 5.146 sec/iter
Epoch: 99 | Batch: 011 / 018 | Total loss: 3.631 | Reg loss: 0.024 | Tree loss: 3.631 | Accuracy: 0.091797 | 5.146 sec/iter
Epoch: 9

In [31]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [33]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 12.0


# Extract Rules

# Accumulate samples in the leaves

In [34]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 4096


In [35]:
method = 'greedy'

In [36]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [37]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

============== Pattern 1 ==============
============== Pattern 2 ==============
============== Pattern 3 ==============
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
============== Pattern 8 ==============
============== Pattern 9 ==============
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
============== Pattern 17 ==============
============== Pattern 18 ==============
============== Pattern 19 ==============
============== Pattern 20 ==============
============== Pattern 21 ==============
============== Pattern 22 ==============
============== Pattern 23 ==============
============== Pattern 24 ==============
============== Pattern 25

/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))


============== Pattern 296 ==============
============== Pattern 297 ==============
============== Pattern 298 ==============
============== Pattern 299 ==============
============== Pattern 300 ==============
============== Pattern 301 ==============
============== Pattern 302 ==============
============== Pattern 303 ==============
============== Pattern 304 ==============
============== Pattern 305 ==============
============== Pattern 306 ==============
============== Pattern 307 ==============
============== Pattern 308 ==============
============== Pattern 309 ==============
============== Pattern 310 ==============
============== Pattern 311 ==============
============== Pattern 312 ==============
============== Pattern 313 ==============
============== Pattern 314 ==============
============== Pattern 315 ==============
============== Pattern 316 ==============
============== Pattern 317 ==============
============== Pattern 318 ==============
============== Pattern 319 =======

============== Pattern 1085 ==============
============== Pattern 1086 ==============
============== Pattern 1087 ==============
============== Pattern 1088 ==============
============== Pattern 1089 ==============
============== Pattern 1090 ==============
============== Pattern 1091 ==============
============== Pattern 1092 ==============
============== Pattern 1093 ==============
============== Pattern 1094 ==============
============== Pattern 1095 ==============
============== Pattern 1096 ==============
============== Pattern 1097 ==============
============== Pattern 1098 ==============
============== Pattern 1099 ==============
============== Pattern 1100 ==============
============== Pattern 1101 ==============
============== Pattern 1102 ==============
============== Pattern 1103 ==============
============== Pattern 1104 ==============
============== Pattern 1105 ==============
============== Pattern 1106 ==============
============== Pattern 1107 ==============
===========

============== Pattern 1487 ==============
============== Pattern 1488 ==============
============== Pattern 1489 ==============
============== Pattern 1490 ==============
============== Pattern 1491 ==============
============== Pattern 1492 ==============
============== Pattern 1493 ==============
============== Pattern 1494 ==============
============== Pattern 1495 ==============
============== Pattern 1496 ==============
============== Pattern 1497 ==============
============== Pattern 1498 ==============
============== Pattern 1499 ==============
============== Pattern 1500 ==============
============== Pattern 1501 ==============
============== Pattern 1502 ==============
============== Pattern 1503 ==============
============== Pattern 1504 ==============
============== Pattern 1505 ==============
============== Pattern 1506 ==============
============== Pattern 1507 ==============
============== Pattern 1508 ==============
============== Pattern 1509 ==============
===========

============== Pattern 2227 ==============
============== Pattern 2228 ==============
============== Pattern 2229 ==============
============== Pattern 2230 ==============
============== Pattern 2231 ==============
============== Pattern 2232 ==============
============== Pattern 2233 ==============
============== Pattern 2234 ==============
============== Pattern 2235 ==============
============== Pattern 2236 ==============
============== Pattern 2237 ==============
============== Pattern 2238 ==============
============== Pattern 2239 ==============
============== Pattern 2240 ==============
============== Pattern 2241 ==============
============== Pattern 2242 ==============
============== Pattern 2243 ==============
============== Pattern 2244 ==============
============== Pattern 2245 ==============
============== Pattern 2246 ==============
============== Pattern 2247 ==============
============== Pattern 2248 ==============
============== Pattern 2249 ==============
===========

============== Pattern 2638 ==============
============== Pattern 2639 ==============
============== Pattern 2640 ==============
============== Pattern 2641 ==============
============== Pattern 2642 ==============
============== Pattern 2643 ==============
============== Pattern 2644 ==============
============== Pattern 2645 ==============
============== Pattern 2646 ==============
============== Pattern 2647 ==============
============== Pattern 2648 ==============
============== Pattern 2649 ==============
============== Pattern 2650 ==============
============== Pattern 2651 ==============
============== Pattern 2652 ==============
============== Pattern 2653 ==============
============== Pattern 2654 ==============
============== Pattern 2655 ==============
============== Pattern 2656 ==============
============== Pattern 2657 ==============
============== Pattern 2658 ==============
============== Pattern 2659 ==============
============== Pattern 2660 ==============
===========

============== Pattern 3582 ==============
============== Pattern 3583 ==============
============== Pattern 3584 ==============
============== Pattern 3585 ==============
============== Pattern 3586 ==============
============== Pattern 3587 ==============
============== Pattern 3588 ==============
============== Pattern 3589 ==============
============== Pattern 3590 ==============
============== Pattern 3591 ==============
============== Pattern 3592 ==============
============== Pattern 3593 ==============
============== Pattern 3594 ==============
============== Pattern 3595 ==============
============== Pattern 3596 ==============
============== Pattern 3597 ==============
============== Pattern 3598 ==============
============== Pattern 3599 ==============
============== Pattern 3600 ==============
============== Pattern 3601 ==============
============== Pattern 3602 ==============
============== Pattern 3603 ==============
============== Pattern 3604 ==============
===========

============== Pattern 4082 ==============
============== Pattern 4083 ==============
============== Pattern 4084 ==============
============== Pattern 4085 ==============
============== Pattern 4086 ==============
============== Pattern 4087 ==============
============== Pattern 4088 ==============
============== Pattern 4089 ==============
============== Pattern 4090 ==============
============== Pattern 4091 ==============
============== Pattern 4092 ==============
============== Pattern 4093 ==============
============== Pattern 4094 ==============
============== Pattern 4095 ==============
============== Pattern 4096 ==============
Average comprehensibility: 54.7666015625
std comprehensibility: 3.3709781843513253
var comprehensibility: 11.363493919372559
minimum comprehensibility: 48
maximum comprehensibility: 68
